## Proses Scraping

In [65]:
# !pip install google-play-scraper

In [66]:
# Memanggil library untuk melakukan scraping
from google_play_scraper import Sort, reviews
from google_play_scraper import app

import pandas as pd
import numpy as np

In [67]:
# mulai scraping
result, continuation_token = reviews(
    'com.strava', #masukkan link setelah kata 'id'
    lang='id', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
    count=2000, # defaults to 100
    filter_score_with=None # defaults to None(means all score)
)

In [68]:
# membuat Dataframe dari hasil scraping
df = pd.DataFrame(np.array(result),columns=['review'])
df = df.join(pd.DataFrame(df.pop('review').tolist()))

In [69]:
# mengekspor dataset
df.to_csv("Review Strava-Google Play Store.csv", index=False, encoding='utf-8')

# Case Folding

In [76]:
# Membaca Dataset
file_path = "Review Strava-Google Play Store.csv"
df = pd.read_csv(file_path)

display(df.head())

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,6ae8bf4a-82f2-49fd-8aee-127d9f8718a6,Ichaa Ca,https://play-lh.googleusercontent.com/a/ACg8oc...,kecewa bangat sama aplikasi ini uang aku di se...,1,0,431.10,2025-10-08 11:14:29,NaN,NaN,431.10
1,add17956-9870-44d0-9c99-f5e7f4dd2905,Rahmat Hidayatullah,https://play-lh.googleusercontent.com/a-/ALV-U...,aplikasi yg hanya menguntungkan pengguna iPhon...,2,0,431.10,2025-10-08 09:24:57,NaN,NaN,431.10
2,e1d90d97-8a14-4625-8e14-08b3e077c5b5,Aprlianti nurmia,https://play-lh.googleusercontent.com/a/ACg8oc...,taiiii babi kasih kembali uang saya kgkgbabiiiii,1,0,431.10,2025-10-08 07:19:08,NaN,NaN,431.10
3,ceac6192-4815-448f-beda-a397e209a489,Bagus Dwi Pribadi,https://play-lh.googleusercontent.com/a-/ALV-U...,mantab,3,0,431.10,2025-10-08 02:58:15,NaN,NaN,431.10
4,d2720434-8fbe-48ac-ab84-db162f8235ec,Catriyan Dimas,https://play-lh.googleusercontent.com/a-/ALV-U...,rekomend bet buat olahraga gak akan nyesel pak...,5,0,424.12,2025-10-08 02:21:23,NaN,NaN,424.12


In [77]:
display(df[['userName', 'content', 'score']].head())

,userName,content,score
0,Ichaa Ca,kecewa bangat sama aplikasi ini uang aku di se...,1
1,Rahmat Hidayatullah,aplikasi yg hanya menguntungkan pengguna iPhon...,2
2,Aprlianti nurmia,taiiii babi kasih kembali uang saya kgkgbabiiiii,1
3,Bagus Dwi Pribadi,mantab,3
4,Catriyan Dimas,rekomend bet buat olahraga gak akan nyesel pak...,5


In [78]:
# 4. Mengecek kolom yang berisi teks review
print("\nNama Kolom di Dataset:")
print(df.columns)


Nama Kolom di Dataset:
Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')


In [79]:
# Melakukan Case Folding
text_column = "content"
df['Case_Folding'] = df[text_column].astype(str).str.lower()

# Menampilkan hasil case folding
display(df[['userName', 'content', 'Case_Folding', 'score']].head())

,userName,content,Case_Folding,score
0,Ichaa Ca,kecewa bangat sama aplikasi ini uang aku di se...,kecewa bangat sama aplikasi ini uang aku di se...,1
1,Rahmat Hidayatullah,aplikasi yg hanya menguntungkan pengguna iPhon...,aplikasi yg hanya menguntungkan pengguna iphon...,2
2,Aprlianti nurmia,taiiii babi kasih kembali uang saya kgkgbabiiiii,taiiii babi kasih kembali uang saya kgkgbabiiiii,1
3,Bagus Dwi Pribadi,mantab,mantab,3
4,Catriyan Dimas,rekomend bet buat olahraga gak akan nyesel pak...,rekomend bet buat olahraga gak akan nyesel pak...,5


In [80]:
df = df.rename(columns={'Case_Folding': 'review'})

In [81]:
# Menyimpan hasil ke file baru
output_path = "Review_Strava_CaseFolding.csv"
df[['userName', 'review', 'score', 'at', 'appVersion']].to_csv(output_path, index=False)